# Part One - Finding the Key from a Synthetic Power Model

In [1]:
import numpy as np
import sys

This part aims to generate for a particular key (`{0x2b, 0x7e, 0x15,0x16,0x28,0xae, 0xd2,0xa6,0xab, 0xf7,0x15,0x88,0x09,0xcf, 0x4f, 0x3c}` as shown in the attached AES_GenPowerProfile.c file) A collection of power traces that contain for each message with 32 bytes

* The Ciphertext can be found in the CIPHERFILE.dat file on the site
* In the state at the entrance to round 9, you can calculate using inverse pomegranates in the DoM_actual_wrapper.py file (requires adjustments)
* It is required to create two tables, one for calculating the power based on HW and the other while relying on HD
    * HW - how many "1 "s are in the state to end each Round
    * HD - how many bits were changed between the state at the end of each stage, compared to the value of the state in the previous stage (the initial state (not in the table) is the input)
    
At the end of this stage you have "in your hands" two matrices with the help of which we can predict the key with which we have encrypted all the messages (place the same key)

In [5]:
CIPHERFILE_PATH = "./Resources/CIPHERFILE.dat"
SAMPLEFILE_PATH = "./Resources/SAMPLEFILE.dat"
SAMPLEFILE_HD_PATH = "./Resources/SAMPLEFILE_HD.dat"
SAMPLEFREQFILE_PATH = "./Resources/SAMPLEFREQFILE.dat"

In [9]:
cipher = []
with open(CIPHERFILE_PATH,"r") as fp: 
    for line in fp: 
        cipher += [line.split()]
display(len(cipher))

5000

In [10]:
samples = []        
with open(SAMPLEFILE_PATH,"r") as fp: 
    for line in fp: 
        samples += [np.array([int(i) for i in line.split()])]
display(len(samples))

256

In [11]:
samples_hd = []
with open(SAMPLEFILE_HD_PATH,"r") as fp:
    for line in fp:
        samples_hd += [np.array([int(i) for i in line.split()])]
display(len(samples_hd))

FileNotFoundError: [Errno 2] No such file or directory: './Resources/SAMPLEFILE_HD.dat'

In [12]:
sample_freqs = []
with open(SAMPLEFREQFILE_PATH,"r") as fp: 
    for line in fp: 
        sample_freqs += [int(line.split()[0])]
display(len(sample_freqs))

256

After creating the tables, we will begin the "prediction" process of the key, using the DOM method learned in class. The method works as follows (suppose here we want to guess the value of the first house of the key, using the right bit (LSB of the house)

* For each house in Ciphertext we will go over all the possible values ​​of the selected house of the developer
    * Using the inverse functions (also given in the file) calculate what was the value of the state which was used as input to the phase
    * Based on the value of select bit, suppose we have chosen the LSB of the house, we will sum their power value, at their absolute value, or to bin0 or bin1. For all 12 sampling points
    * At the end of the transition on all the encrypted messages, we will get two vectors which should normalize them according to the number of messages used in each bin
    * The normal vectors we subtract from each other and the absolute value of the differences, at the various points, is used to select the key, using one of the following three cryorions
        * The key used to create the largest difference for the selected house
        * The key used to create the largest difference for one of the sampling points (vector point)
        * The key used to create the large average difference
        

In [14]:
InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

# TARGET_BYTE = 4
BIT_MASK = 2**5

## Find key by Hamming Weight


In [15]:
key = -1
_max = 0
for key_guess in range(256):
    p_0 = np.zeros(12)
    n_0 = 0
    p_1 = np.zeros(12)
    n_1 = 0
    for idx in range(256):
        select_bit = 0 if (InvSbox[key_guess ^ idx] & BIT_MASK == 0) else 1
        if select_bit == 0:
            p_0 += samples[idx]
            n_0 += sample_freqs[idx]
        else:
            p_1 += samples[idx]
            n_1 += sample_freqs[idx]

    if (n_0 != 0 and n_1 != 0):
        dom = np.abs(p_0/n_0 - p_1/n_1)
        dom_max = np.sum(dom)
        # print(dom_max)

        if (dom_max > _max):
            _max = dom_max
            key = hex(key_guess)

    else:
        print(str(idx) + " Failed")

key_hw = key
print(key_hw)

0xc9


## Find key by Hamming Distance

In [17]:
key = -1
_max = 0
for key_guess in range(256):
    p_0 = np.zeros(11)
    n_0 = 0
    p_1 = np.zeros(11)
    n_1 = 0
    for idx in range(256):
        select_bit = 0 if ((InvSbox[key_guess ^ idx] & BIT_MASK) ^ (idx & BIT_MASK) == 0) else 1
        if select_bit == 0:
            p_0 += samples_hd[idx]
            n_0 += sample_freqs[idx]
        else:
            p_1 += samples_hd[idx]
            n_1 += sample_freqs[idx]

    if (n_0 != 0 and n_1 != 0):
        dom = np.abs(p_0/n_0 - p_1/n_1)
        dom_max = np.max(dom)

        if (dom_max > _max):
            _max = dom_max
            key = hex(key_guess)

    else:
        print(str(idx) + " Failed")

key_hd = key
print(key_hd)

IndexError: list index out of range